In [ ]:
# !pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import gc
import numpy as np

import torch
import torch.nn as nn
from transformers import get_scheduler
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_outputs import CausalLMOutput
from datasets import load_dataset

from huggingface_hub import login

In [ ]:
HF_TOKEN = ""
login(HF_TOKEN)

In [ ]:
MODEL_NAME = "fxmarty/small-llama-testing"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch.nn.functional as F

def preprocess_function(examples, virtual_token_embeddings, virtual_token_ids):
    prompt_template = "{context}\nQuestion: {question}\nAnswer:"
    inputs = [
        prompt_template.format(context=c, question=q)
        for c, q in zip(examples["context"], examples["question"])
    ]

    # print(examples['answers'])
    answers = [answer['text'][0] for answer in examples["answers"]]  # First answer as the target

    # Tokenize inputs and labels
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(answers, max_length=128, truncation=True, padding="max_length", return_tensors="pt")["input_ids"]

    for key in model_inputs.keys():
        model_inputs[key] = model_inputs[key].to(model.device)
    labels = labels.to(model.device)

    # virtual_token_embeddings = virtual_token_embeddings.to(model.device)
    # virtual_token_ids = virtual_token_ids.to(model.device)

    # # Include virtual tokens
    # model_inputs["input_embeddings"], model_inputs["attention_mask"] = add_virtual_tokens(
    #     model_inputs["input_ids"], model_inputs["attention_mask"], virtual_token_embeddings, virtual_token_ids
    # )
    model_inputs["labels"] = labels

    model_inputs["labels"] = F.pad(
        model_inputs["labels"],
        (0, model_inputs["input_ids"].size(1) - model_inputs["labels"].size(1)),
        value=-100
    )

    # for key in model_inputs.keys():
    #     # model_inputs[key] = model_inputs[key].to(model.device)
    #     print(key, model_inputs[key].shape)

    return model_inputs

def preprocess_v1(batch, *args, **kwargs):
    # Tokenize inputs first
    inputs = [
        "{context}\nQuestion: {question}\nAnswer:".format(context=c, question=q)
        for c, q in zip(batch["context"], batch["question"])
    ]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Move inputs to the correct device
    model_inputs['input_ids'] = model_inputs['input_ids'].to(kwargs["device"])
    model_inputs['attention_mask'] = model_inputs['attention_mask'].to(kwargs["device"])

    # Tokenize answers
    answers = [answer['text'][0] for answer in batch["answers"]]
    model_inputs["labels"] = tokenizer(
        answers,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )['input_ids'].to(kwargs["device"])

    return model_inputs

def original_init(embedding_layer, mean=0.0, std_dev=0.02):
    # print(embedding_layer)
    nn.init.normal_(embedding_layer, mean=mean, std=std_dev)


def gaussian_fitted_init(embedding_layer, prev_embeddings):
    mean_vector = torch.from_numpy(np.mean(prev_embeddings, axis=0)).to(torch.float32)
    cov_matrix = torch.from_numpy(np.cov(prev_embeddings.T)).to(torch.float32)

    mvn = torch.distributions.MultivariateNormal(mean_vector, covariance_matrix=cov_matrix)

    with torch.no_grad():
        sampled_weights = mvn.sample(embedding_layer.shape)
        embedding_layer = nn.Parameter(sampled_weights)

class DeepPromptTunedModel(nn.Module):
    def __init__(
            self,
            base_model_name,
            preprocess_function,
            num_virtual_tokens=20,
            num_layers=None
    ):
        super().__init__()
        # Load base model
        self.base_model = AutoModelForCausalLM.from_pretrained(base_model_name, token=HF_TOKEN)

        # Determine number of layers if not specified
        if num_layers is None:
            num_layers = len(self.base_model.model.layers)

        # Create virtual token parameters for each layer
        self.virtual_token_parameters = nn.ParameterList([
            nn.Parameter(torch.randn(num_virtual_tokens, self.base_model.config.hidden_size))
            for _ in range(num_layers)
        ])

        self.preprocess_function = preprocess_function
        self.num_virtual_tokens = num_virtual_tokens
        self.device = self.base_model.device

        # Freeze base model parameters
        for parameter in self.base_model.parameters():
            parameter.requires_grad = False

        # Make virtual token parameters trainable
        for param in self.virtual_token_parameters:
            param.requires_grad = True

    def init_virtual_token_parameters(
            self,
            init_function=None,
            init_function_kwargs=None
    ):
        """
        Initialize virtual token parameters

        Args:
            init_function: Initialization method
            init_function_kwargs: Dictionary of initialization arguments
        """
        if init_function is None:
            # Default to Xavier initialization
            init_function = torch.nn.init.xavier_uniform_

        init_function_kwargs = init_function_kwargs or {}

        for layer_params in self.virtual_token_parameters:
            init_function(layer_params, **init_function_kwargs)
            layer_params.to(self.device)

    def add_virtual_tokens(self, input_ids, attention_mask):
        """
        Add virtual tokens to input at each layer

        Returns:
            Modified input embeddings and attention mask
        """
        batch_size = input_ids.size(0)

        # Get input embeddings
        input_embeddings = self.base_model.get_input_embeddings()(input_ids)

        # Prepare attention mask for virtual tokens
        virtual_attention = torch.ones(
            (batch_size, self.num_virtual_tokens),
            dtype=torch.long
        ).to(input_ids.device)

        # Extend embeddings and attention mask with virtual tokens at each layer
        layer_input_embeddings = []
        layer_attention_masks = []

        for layer_params in self.virtual_token_parameters:
            # Expand virtual token parameters for the batch
            layer_virtual_embeddings = layer_params.unsqueeze(0).expand(batch_size, -1, -1)

            # Concatenate virtual and input embeddings
            layer_current_embeddings = torch.cat([layer_virtual_embeddings, input_embeddings], dim=1)
            layer_current_attention = torch.cat([virtual_attention, attention_mask], dim=1)

            layer_input_embeddings.append(layer_current_embeddings)
            layer_attention_masks.append(layer_current_attention)

        return layer_input_embeddings, layer_attention_masks

    def forward(self, batch):
        """
        Forward pass with deep prompt tuning

        Args:
            batch: Input batch of data

        Returns:
            Model outputs
        """
        # Preprocess and tokenize inputs
        tokenized_batch = self.preprocess_function(batch, self.virtual_token_parameters, None, device=self.device)

        # for key in tokenized_batch.keys():
        #     tokenized_batch[key] = tokenized_batch[key].to(model.device)

        # print(tokenized_batch)

        # Add virtual tokens at each layer
        layer_input_embeddings, layer_attention_masks = self.add_virtual_tokens(
            tokenized_batch['input_ids'],
            tokenized_batch['attention_mask']
        )

        # Prepare initial states
        hidden_states = layer_input_embeddings[0]

        # Store intermediate outputs for potential use
        all_hidden_states = []

        # Custom forward pass through model layers
        for i, layer in enumerate(self.base_model.model.layers):
            # Prepare position ids for the expanded input
            position_ids = torch.arange(
                0, layer_input_embeddings[i].size(1),
                dtype=torch.long,
                device=layer_input_embeddings[i].device
            ).unsqueeze(0).repeat(layer_input_embeddings[i].size(0), 1)

            # Prepare causal mask for attention
            batch_size, seq_length = layer_input_embeddings[i].size(0), layer_input_embeddings[i].size(1)
            causal_mask = torch.zeros(
                (batch_size, 1, seq_length, seq_length),
                dtype=torch.bool,
                device=layer_input_embeddings[i].device
            )
            causal_mask = torch.triu(torch.ones_like(causal_mask[0], dtype=torch.bool), diagonal=1)

            # Expand the causal mask to match batch size
            causal_mask = causal_mask.unsqueeze(0).expand(batch_size, 1, seq_length, seq_length)

            # Combine causal mask with the existing attention mask
            if layer_attention_masks[i] is not None:
                extended_attention_mask = layer_attention_masks[i].unsqueeze(1).unsqueeze(2)
                extended_attention_mask = extended_attention_mask.to(dtype=torch.bool)
                causal_mask = causal_mask | (~extended_attention_mask)

            # Layer forward pass
            layer_outputs = layer(
                hidden_states=layer_input_embeddings[i],
                attention_mask=causal_mask,
                position_ids=position_ids
            )

            hidden_states = layer_outputs[0]
            all_hidden_states.append(hidden_states)

        # Final layer processing
        logits = self.base_model.lm_head(hidden_states)

        # Compute loss
        loss = None
        if tokenized_batch['labels'] is not None:
            # Pad labels to match input embedding length
            labels = F.pad(
                tokenized_batch['labels'],
                (0, layer_input_embeddings[0].size(1) - tokenized_batch['labels'].size(1)),
                value=-100
            )
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                labels.view(-1),
                ignore_index=-100
            )

        return CausalLMOutput(**{
            'loss': loss,
            'logits': logits,
            'hidden_states': all_hidden_states
        })

    def to(self, device):
        """
        Move model and all components to specified device
        """
        self.base_model.to(device)
        for layer_params in self.virtual_token_parameters:
            layer_params = layer_params.to(device)
        self.device = device
        return self

model_name = MODEL_NAME

# Example initialization
model = DeepPromptTunedModel(
    base_model_name=model_name,
    # preprocess_function=preprocess_function,
    preprocess_function=preprocess_v1,
    num_virtual_tokens=20,  # Number of virtual tokens per layer
)

# Initialize virtual token embeddings
# You can use custom initialization like before
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
tokenizer.pad_token_id = tokenizer.eos_token_id

vocab = list(tokenizer.get_vocab().keys())

N = 20000
random_indices = np.random.choice(len(vocab), N, replace=False)
prev_embeddings = model.base_model.get_input_embeddings().weight.detach().cpu().numpy()[random_indices]

model.init_virtual_token_parameters(
    # init_function=original_init,
    init_function=gaussian_fitted_init,
    init_function_kwargs={"prev_embeddings": prev_embeddings}
)

In [ ]:
model.virtual_token_parameters

ParameterList(
    (0): Parameter containing: [torch.float32 of size 20x256]
    (1): Parameter containing: [torch.float32 of size 20x256]
)

In [ ]:
trainable_params = 0
total_params = 0
for parameter in model.parameters():
  if parameter.requires_grad:
    trainable_params += 1
  total_params += 1

print(f"Trainable Params: {(trainable_params / total_params) * 100}%")

Trainable Params: 8.695652173913043%


In [ ]:
from torch.utils.data import Dataset

class PromptTuningDataset(Dataset):
    def __init__(self, dataset, preprocess_function, virtual_token_embeddings, virtual_token_ids):
        super().__init__()
        self.dataset = dataset
        self.preprocess_function = preprocess_function
        self.virtual_token_embeddings = virtual_token_embeddings
        self.virtual_token_ids = virtual_token_ids

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        item = self.preprocess_function(item, self.virtual_token_embeddings, self.virtual_token_ids)
        return item

class PromptTuningCollateClass:
    def __call__(self, batch):
        return {
            "context": [item["context"] for item in batch],
            "question": [item["question"] for item in batch],
            "answers": [item["answers"] for item in batch]
        }

In [ ]:
def custom_forward(model, input_embeddings, attention_mask, labels=None):
    outputs = model(
        inputs_embeds=input_embeddings,
        attention_mask=attention_mask,
        labels=labels,
    )
    return outputs

In [ ]:
# Preprocess data function (assuming you have a preprocess function defined)
# train_dataset = dataset["train"].map(preprocess_function, batched=True)
# val_dataset = dataset["validation"].map(preprocess_function, batched=True)
# Load SQuAD dataset as an example
dataset = load_dataset("squad")

train_dataset = dataset['train']
# PromptTuningDataset(
#     dataset["train"],
#     preprocess_function,
#     virtual_token_embeddings,
#     virtual_token_ids
# )
val_dataset = dataset['validation']
# PromptTuningDataset(
#     dataset["validation"],
#     preprocess_function,
#     virtual_token_embeddings,
#     virtual_token_ids
# )

collate = PromptTuningCollateClass()

# Using smaller batch sizes for memory efficiency
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, pin_memory=False, collate_fn=collate)
val_loader = DataLoader(val_dataset, batch_size=2, pin_memory=False, collate_fn=collate)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(
    list(model.virtual_token_parameters.parameters()), lr=5e-3
)
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_training_steps=num_training_steps,
    num_warmup_steps=0
)

# Training loop
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Enable mixed precision training for better memory usage (if using a compatible GPU)
# scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None
scaler = None

In [ ]:
model.device

'cpu'

In [ ]:
model.base_model.device

device(type='cpu')

In [ ]:
def get_grad_norm(model, norm_type=2):
    parameters = model.parameters()
    if isinstance(parameters, torch.Tensor):
        parameters = [parameters]
    parameters = [p for p in parameters if p.grad is not None]
    total_norm = torch.norm(
        torch.stack(
            [torch.norm(p.grad.detach(), norm_type).cpu() for p in parameters]
        ),
        norm_type,
    )
    return total_norm.item()

In [ ]:
model.virtual_token_parameters

ParameterList(
    (0): Parameter containing: [torch.float32 of size 20x256]
    (1): Parameter containing: [torch.float32 of size 20x256]
)

In [ ]:
import wandb

import os
os.environ['WANDB_API_KEY'] = "ddf693372fa6005234881e204454cf3757d91f84"
os.environ["WANDB_MODE"] = "online"

In [ ]:
train_loader.batch_size

2

In [ ]:
wandb.init(
    project="Bayesian ML Final Project",
    name="mini_llama_dp_prmpt_tune_gaussian_ini",
    config={
        "batch_size": train_loader.batch_size
    }
)

wandb.watch(model, log=None, log_freq=10)

In [ ]:
from tqdm.notebook import tqdm

# virtual_token_embeddings_history = [model.virtual_token_parameters.weight.detach().clone()]

N_EPOCHS = 10
LEN_EPOCH = 10
LEN_VAL_EPOCH = 10


for epoch in range(N_EPOCHS):  # Number of epochs
    model.train()
    epoch_train_loss = 0  # Accumulate training loss

    # if epoch == 0:
    #     wandb.log({
    #         "virtual_token_embeddings": wandb.Histogram(
    #             model.virtual_token_embeddings.weight.cpu().detach().numpy().copy()
    #         )
    #     }, step=0)

    for step, batch in tqdm(enumerate(train_loader), total=LEN_EPOCH):
        optimizer.zero_grad()

        # Move data to the appropriate device
        # input_embeddings = batch["input_embeddings"].to(device)
        # attention_mask = batch["attention_mask"].to(device)
        # labels = batch["labels"].to(device)

        # Mixed precision training: use autocast for FP16 precision
        if scaler and False:
            with torch.cuda.amp.autocast():
                outputs = model(
                    inputs_embeds=input_embeddings,
                    attention_mask=attention_mask,
                    labels=labels,
                )
                loss = outputs.loss
        else:
            outputs = model(batch)
            loss = outputs.loss

        # Backpropagation and optimization
        if scaler and False:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        lr_scheduler.step()

        # Accumulate loss for logging
        epoch_train_loss += loss.item()

        # Log metrics to wandb
        wandb.log({
            "train_loss": loss.item(),
            "learning_rate": optimizer.param_groups[0]['lr'],
            "grad_norm": get_grad_norm(model.virtual_token_parameters)
        }, step=step + epoch * LEN_EPOCH)

        # Print training loss for monitoring
        # print(f"Epoch {epoch+1}, Step {step+1}/{len(train_loader)}, Loss: {loss.item()}")

        if step >= LEN_EPOCH:
            break

    # virtual_token_embeddings_history.append(model.virtual_token_parameters.weight.detach().clone())


    # Validation (optional, and with no gradients)
    model.eval()
    epoch_val_loss = 0  # Accumulate validation loss
    with torch.no_grad():
        for val_step, batch in enumerate(val_loader):
            # input_embeddings = batch["input_embeddings"].to(device)
            # attention_mask = batch["attention_mask"].to(device)
            # labels = batch["labels"].to(device)

            outputs = model(batch)
            val_loss = outputs.loss
            epoch_val_loss += val_loss.item()

            if val_step >= LEN_VAL_EPOCH:
                break

        # Log validation metrics to wandb
        # wandb.log({
        #     "val_loss": epoch_val_loss / LEN_VAL_EPOCH,
        #     "virtual_token_embeddings": wandb.Histogram(
        #         model.virtual_token_embeddings.weight.cpu().detach().numpy().copy()
        #     )
        # }, step=(epoch + 1) * LEN_VAL_EPOCH)

        print(f"Epoch {epoch+1}, Validation Loss: {epoch_val_loss / LEN_VAL_EPOCH}")

    # Log epoch-level metrics
    wandb.log({
        "epoch": epoch + 1,
        "epoch_train_loss": epoch_train_loss / len(train_loader),
        "epoch_val_loss": epoch_val_loss / len(val_loader),
    })

    # Clear CUDA cache to reduce memory fragmentation after each epoch
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

# Finish the wandb run
wandb.finish()

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1, Validation Loss: 11.431087207794189


  0%|          | 0/10 [00:00<?, ?it/s]

wandb: WARNING Tried to log to step 10 that is less than the current step 11. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Epoch 2, Validation Loss: 11.392326641082764


...

Epoch 9, Validation Loss: 11.123904800415039


  0%|          | 0/10 [00:00<?, ?it/s]

wandb: WARNING Tried to log to step 90 that is less than the current step 91. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Epoch 10, Validation Loss: 11.08598289489746


epoch,▁▂▃▃▄▅▆▆▇█
epoch_train_loss,█▇▆▆▅▄▃▃▂▁
epoch_val_loss,█▇▆▆▅▄▃▃▂▁
grad_norm,█████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁
learning_rate,████▇▇▇▇▇▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁
train_loss,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
epoch,10
epoch_train_loss,0.00254
epoch_val_loss,0.02098
grad_norm,0.01087
learning_rate,0.005
